In [0]:
%sql
use strange;
CREATE TABLE person (
    id STRING,
    name STRING,
    ADDR STRING,
    DATE STRING,
    Active BOOLEAN    
);

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("SCD2-Incremental").getOrCreate()

# Existing SCD2 Table
existing_data = [
    ("1","A","Pune","2025-07-01", True),
    ("2","B","Pune","2025-07-15", True),   
    ("3","C","Pune","2025-07-15", True),# currently active
]
columns = ["ID","NAME","ADDR","DATE","Active"]
df_existing = spark.createDataFrame(existing_data, columns)

print("Current SCD2 Table:")
df_existing.createOrReplaceTempView("person_data")
df_existing.show()

# New incoming data
new_data = [("4","D","Goa","2025-07-25"),
            ("5","E","Pune","2025-08-25"),
            ("3","C","Mumbai","2025-08-25"),
            ("2","B","Pune","2025-07-15"),
            ]
columns_new = ["ID","NAME","ADDR","DATE"]
df_new = spark.createDataFrame(new_data, columns_new)

print("New Data:")
df_new.show()


In [0]:
%sql
--insert into person  
--select * from person_data


In [0]:
%sql
use strange;
select * from person

### 1.From Target table remove audit columns

In [0]:
df_person = spark.read.table("strange.person")
df_target  = df_person.select("ID","NAME","ADDR","DATE") 
df_target.show()

### 2.use xxhash64 compare source and target df excluding audit


In [0]:
from pyspark.sql.functions import xxhash64

#df_target= df_target.withColumn("hash64",xxhash64(col("ID"),col("NAME"),col("ADDR"),col("DATE")))
df_target= df_target.withColumn("hash64",xxhash64("ID","NAME","ADDR","DATE"))
#df_new= df_new.withColumn("hash64",xxhash64(col("ID"),col("NAME"),col("ADDR"),col("DATE")))

df_target.show()
#df_new.show()

###  3. perform left anti join to remove same records which are not changed exmaple id 2 no change


In [0]:
df_result = df_new.join(df_target,"hash64","left_anti").drop("hash64")
df_result.show()

### 4.Add audit columns to result df

In [0]:
from pyspark.sql.functions import current_timestamp,to_timestamp

df_result = df_result.withColumn("Active",lit(True))
 # .withColumn("start_time",to_timestamp(current_timestamp(),"yyyy-MM-dd HH:mm:ss"))\
 #   .withColumn("End_time", to_timestamp(lit("9999-12-31 00:00:00"),"yyyy-MM-dd HH:mm:ss"))

df_result.createOrReplaceTempView("person_result")
df_result.show(truncate = False )

### 5. Write result into target table

In [0]:
%sql
insert into person  
select * from person_result

### Generate Row number partition by id order by date desc

In [0]:

df_final = spark.sql('''
with cte as (
select id,name,addr,date, row_number() over(partition by id order by date desc) as rnk,
(case when  rnk >1 then False else Active end) as Active
from person
)
select id,name,addr,date,active from cte 
''')
df_final.show(truncate = False) 

### for updating start date and end date of old records us lag 

In [0]:
windspec = Window.partitionBy("id").orderBy("date").desc()
